In [15]:
from datetime import datetime
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd

In [16]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [17]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
)

In [18]:
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=10000,
  where=[
    snapshot.Proposal.space == 'olympusdao.eth',
    snapshot.Proposal.state == 'closed',
  ]
)

In [19]:
proposals_snapshots = sg.query_df([
  proposals.title,
  proposals.id,
  proposals.body
])

In [20]:
proposals_choices = sg.query(proposals.choices)

In [21]:
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_a', 'option_b', 'option_c', 'option_d', 'option_e','option_f','option_g'])

In [22]:
olympus_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

In [23]:
olympus_governance_view

,proposals_title,proposals_id,proposals_body,option_a,option_b,option_c,option_d,option_e,option_f,option_g
0,OIP-94: Interim Ranged Stability Policy Levers,0x6413984c8f1076e3f46f196810ba51781f25f4d2f1eb...,abipup\n \n 2 days ago\n\nSumma...,Inverse bonds & adjust liquidity,Do nothing,None,None,None,None,None
1,OIP-93: Mint and Sync,0xf0e62d0a9eeb47a295bab9a8d0cc84c0d4de0a8cb000...,Summary:\n\nEnable minting into liquidity pool...,"1. Implement, no change to RR","2. Implement, change RR",3. Do not implement,None,None,None,None
2,TAP-12 Pilot Deposit on Tribe Turbo,0x7d191f4a7f2ea3d138c07f73d568b93c96fc5ffce267...,Summary\n\nSeek authority from the community t...,Approve Pilot Deposit,Do not approve pilot deposit,None,None,None,None,None
3,TAP-11 Ribbon Finance Whitelist,0xe52a4af6307bea40a34f2eaf9a93f11c8abf382020cc...,Summary: Seek authority from the community to ...,Allow Ribbon Finance,Do not allow Ribbon Finance,None,None,None,None,None
4,OIP - 92 : Whitelist FEI as a treasury Reserve...,0xe9aa47f39bc5ace6ad5a19c08007a3683386e19c1390...,Summary\n\nWhitelist FEI as our next reserve a...,Approve FEI in the Treasury,Do not approve FEI in Treasury,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
192,OIP-5: Lower LP Staking Rewards,QmTrwGU99RMqTeeCKy5oW9Le9M2mXHHdNeEWzn7T67watz,Summary: We should decrease LP staking rewards...,For: Option 1,For: Option 2,Against,None,None,None,None
193,Deposit SLP,QmZhSNWAzrDy5KiGSeBsKBXUHW6D4vJnfCuqmdai8LPETd,Summary: The DAO currently holds liquidity tha...,Deposit; keep Onsen,Deposit; no Onsen,Don't deposit,None,None,None,None
194,Loosen Policy,QmZJb2evEjsxtLz92XBuhApWty69fVqUpFHzuVjMjCd3Se,Summary: Let's adjust system parameters away f...,Do it,Don't do it,None,None,None,None,None
195,Add DAI Bonds,QmUC5xa4YaWZBV7fy241svBQ8bHSMksm8kB7avk1WyWv9m,Summary: We can launch bonds using DAI instead...,Add; remove sales,Add; phase out sales,Add; keep sales,Do not add,None,None,None


In [24]:
olympus_governance_view.to_csv('olympus_snapshots_list.csv')

In [66]:
vote_tracker = snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=10000,
    where=[
      snapshot.Vote.proposal == '0x6413984c8f1076e3f46f196810ba51781f25f4d2f1ebee606455929cbbc7d7c8'
    ]
)

In [67]:
voting_snapshots = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice
])

In [68]:
voting_snapshots.to_csv(r'/Users/ianbrillembourg/Downloads/OIPS/OIP94_snapshot.csv', index = False)